In [3]:
# 실습 #1: langchain을 이용하여 사용자가 업로드한 pdf 파일을 근거로 답변하는 ChatGPT 어플리케이션을 작성하시오.

!pip install -qU langchain_community
!pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 79.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 KB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 KB 24.3 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
import os

# 홈 디렉토리 포함된 경로를 안전하게 확장
path = os.path.expanduser("~/work/koshipa-llm-2025-1st/src/exercise/soyeon/day02/소나기.pdf")


In [10]:
loader = PDFPlumberLoader(path)
docs = loader.load()
documents = docs

In [7]:
print(docs[0].metadata)

{'source': '/home/ubuntu/work/koshipa-llm-2025-1st/src/exercise/soyeon/day02/소나기.pdf', 'file_path': '/home/ubuntu/work/koshipa-llm-2025-1st/src/exercise/soyeon/day02/소나기.pdf', 'page': 0, 'total_pages': 7, 'Author': 'MYHOME', 'Creator': 'Hwp 2018 10.0.0.11131', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20211129200550+09'00'", 'ModDate': "D:20211129200550+09'00'", 'PDFVersion': '1.4'}


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

embeddings = OpenAIEmbeddings()
#embeddings = HuggingFaceEmbeddings() # sentence-transformers/all-mpnet-base-v2

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    ).from_loaders([loader])

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

index.query("요약해줘", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'이 글은 소년과 소녀의 일상적인 순간을 묘사하고 있습니다. 소녀는 대추를 따서 소년에게 주며, 제사를 지낼 것이라고 이야기합니다. 소년은 소녀의 어깨를 감싸주고, 비를 피하기 위해 수숫단 속으로 들어가려 합니다. 대화 중에 소년의 가족과 소녀의 가족에 대한 이야기가 오가며, 소녀의 집안이 어려운 상황에 처해 있다는 점이 드러납니다. 전체적으로 소년과 소녀의 관계와 그들의 가족 상황이 엮여 있는 이야기입니다.'

In [9]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os

# 1. OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] 

# 2. PDF 불러오기
path = os.path.expanduser("~/work/koshipa-llm-2025-1st/src/exercise/soyeon/day02/소나기.pdf")
loader = PDFPlumberLoader(path)
documents = loader.load()

# 3. 문서 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

# 4. 임베딩 및 벡터 DB 저장
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

# 5. Retriever QA 체인 구성
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0),
    retriever=retriever,
    return_source_documents=True,
)

# 6. 질문
question = "줄거리를 요약해줘"
result = qa_chain({"query": question})

# 7. 결과 출력
print("📌 답변:\n", result["result"])

📌 답변:
 이 이야기는 소년과 소녀의 짧은 만남과 그들 사이의 감정을 다루고 있습니다. 소년은 소녀와 함께 산을 오르며 꽃을 꺾고, 소녀가 다쳤을 때 송진을 발라주는 등 소녀에게 호감을 보입니다. 소녀는 서울에서 온 아이로, 시골 생활에 적응하지 못하고 심심해합니다. 둘은 함께 시간을 보내며 가까워지지만, 소녀는 결국 병으로 세상을 떠나게 됩니다. 소녀의 죽음은 소년에게 큰 충격을 주며, 소녀의 마지막 소원은 자신이 입던 옷을 입고 묻히는 것이었습니다. 이야기는 소년의 순수한 감정과 소녀의 죽음을 통해 삶의 무상함을 보여줍니다.
